# PHASE 1: Platform Classification using Supervised Learning Methods - Building Training and Test datasets (part 2)

### References:
* https://www.quora.com/What-is-the-best-way-to-create-a-training-set-for-machine-learning/answer/Clem-Wang-1

## Main Python libraries

In [70]:
import os, sys, pathlib
from IPython.display import display, Math, Latex #also '%%latex' magic command
import collections, itertools, operator, re, copy, datetime
import urllib, urllib.request, urllib.parse, dns, ipwhois
import pickle, json, csv, zipfile
import math, random, numpy, scipy, pandas
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import bs4
import nltk, sklearn

#actualcwd = os.getcwd()
#os.chdir(actualcwd)
#print(os.getcwd())

In [2]:
#a simple but not adequate hack to solve:
#-- that this notebook will include any passed module that exists on top of its folder
#-- that my installation of the the nltk will find the correct path to the nlkt_data folder
#
#once this line is run, IT SHOULDN'T RUN AGAIN!, otherwise the cwd will change into something different; 
#it can restored though in different ways, one by using `actualcwd` variable or in some cases shuting down this notebook
#
#
os.chdir('../..')
#print(os.getcwd())
sys.path.append(os.getcwd())
import config.config as config

#print(config.anacondadir)

if os.path.exists(config.anacondadir):
    print('ok')
    nltk.data.path.append(config.anacondadir)

ok


In [3]:
STOP HERE!!

SyntaxError: invalid syntax (<ipython-input-3-bfc7f4b2e6e8>, line 1)

In [4]:
#OBSERVATION!: name of file has been changed to reflect last updated file 
if pathlib.Path(os.getcwd()+'/data/annotatedplatformsphase1_a1.csv').is_file():
    pd_annotated = pandas.read_csv(open(os.getcwd()+'/data/annotatedplatformsphase1_a1.csv', 'r'), sep=';', quotechar="'")
else:
    print('Not path found')

In [5]:
pd_annotated.head(5)

,Unnamed: 0,Unnamed: 0.1,platform,title,description,keywords,htext,params,category_regex,wiki,category
0,0,0,v4-alpha.getbootstrap.com,"\n \n bootstrap · the most popular html, c...","the most popular html, css, and js framework i...",,"Bootstrap is the most popular HTML, CSS, and J...",",/components,/components/breadcrumb/,/migratio...",api|package|framework|librar|stack|licens|addo...,NaN,PACKAGE
1,1,3,jscroll.com,jscroll - a jquery plugin for infinite scrolli...,,,jScroll is a jQuery plugin for infinite scroll...,"/,https://github.com/cubiq/iscroll",api|package|framework|librar|stack|licens|addo...,NaN,PACKAGE
2,2,10,mongoosejs.com,mongoose odm v4.10.5,,,Elegant MongoDB object modeling for Node.js Mo...,"/docs/promises.html,/docs/populate.html,/docs/...",api|package|framework|librar|stack|licens|addo...,NaN,PACKAGE
3,3,34,www.highcharts.com,Interactive JavaScript charts for your webpage...,Highcharts - Interactive JavaScript charts for...,"highcharts, charts, javascript charts, ajax ch...",View demo Get a license,"/samples/data/jsonp.php,/blog/192-use-highchar...",api|package|framework|librar|stack|licens|addo...,NaN,PACKAGE
4,4,41,www.crossorigin.me,crossorigin.me,,,"\n Welcome to crossorigin.me,...","/https://en.wikipedia.org/w/api.php,/,/https:/...",api|package|framework|librar|stack|licens|addo...,NaN,PACKAGE


In [6]:
#OBSERVATION!: name of file has been changed to reflect last updated file 
if pathlib.Path(os.getcwd()+'/data/notatedplatformsphase1_a1.csv').is_file():
    pd_notated = pandas.read_csv(open(os.getcwd()+'/data/notatedplatformsphase1_a1.csv', 'r'), sep=';', quotechar="'")
else:
    print('Not path found')

In [7]:
pd_notated.head(5)

,Unnamed: 0,Unnamed: 0.1,platform,title,description,keywords,htext,params,category,wiki
0,0,0,www.kenneth-truyers.net,my life as a software developer |,blog about development using .net and client-s...,,"Posted on Friday, January 27th, 2017 at 02:42 ...",/2013/04/20/javascript-hoisting-explained/,NaN,NaN
1,1,1,school.scotch.io,courses ― scotch,,,You trust Scotch tutorials and Scotch authors....,"/build-a-nodejs-website/course-introduction,/b...",NaN,NaN
2,2,2,www.imkrisna.com,i made krisna widhiastra | full-stack developer,,,\n\t\t\t\t\t\t\tCurrently working as Applicati...,/blog/2014/02/if-else-vs-if-if-vs-switch-javas...,NaN,NaN
3,3,3,vanilla-js.com,vanilla js,,,The Vanilla JS team maintains every byte of co...,/,NaN,NaN
4,4,4,www.newmediacampaigns.com,new media campaigns | nmc digital agency,NaN,,Congrats to Hutchison PLLC on being named one ...,/blog/how-to-use-json-apis-with-jquery,NaN,NaN


## DATA UPDATING - Important...

The code commented below was used to:
* **Add wikipedia info to the created tables**: for this I used the `scr.wikipedia_extract` module made for this purpose
* **Revise some column naming and use the names for categories** to be used from now on: until now (29-July-17), I was using as categories a regex form I used for the original classification rules; from now I will be using the defined categories found in the `docs` folder of this work

The changes below were done by creating updated files, re-opening that file and update it until the right file was created. Once the information was updated with some information, the respective code was commented because it was not of usage any more.

The code below is kept commented but is shown so other users of this file can have a reference. Also the different intermediate files are kept in the `data` folder but only the required files are called for further analysis. In a more advanced phase a simpler file will be kept.

In [8]:
#import scr.wikipedia_extract

In [9]:
#import imp

In [10]:
#imp.reload(scr.wikipedia_extract)

In [11]:
#pd_annotated['wiki'] = ''
#pd_notated['wiki'] = ''
#scr.wikipedia_extract.getting_wikipedia(pd_annotated)
#scr.wikipedia_extract.getting_wikipedia(pd_notated)

In [12]:
#pd_errortemp = pd_notated.loc[pd_notated['platform'] == 'view-source:http:',]
#scr.wikipedia_extract.getting_wikipedia(pd_errortemp)
#print(pd_errortemp)
#print(pd_notated.iloc[110,])
#url = 'https://en.wikipedia.org/w/api.php?action=query&list=search&format=json&srsearch=JavaScript Objects in Detail'
#req = urllib.request.Request(url)
#try:
#    resp = urllib.request.urlopen(req)
#except urllib.error.HTTPError as e:
#    print(e)

In [13]:
#pd_notated.head(100)

In [14]:
#pd_annotated = pd_annotated.rename(columns={'category':'category_regex'})

In [15]:
#pd_annotated.columns

In [16]:
#categories = pandas.read_csv(open(os.getcwd()+'/docs/category_operationalization.csv', 'r'), sep=':')

In [17]:
#categories

In [18]:
#categories = categories[['category','category_regex']]

In [19]:
#pd_annotated = pandas.merge(pd_annotated, categories, on=['category_regex'])

In [20]:
#pd_annotated.head(5)

In [21]:
#pd_annotated.to_csv(os.getcwd()+'/data/annotatedplatformsphase1_a1.csv', sep=';', quotechar="'")
#pd_notated.to_csv(os.getcwd()+'/data/notatedplatformsphase1_a1.csv', sep=';', quotechar="'")

In [22]:
STOP HERE!

SyntaxError: invalid syntax (<ipython-input-22-d5af98942240>, line 1)

## DATA PREPARATION

In [23]:
import scr.datapreparation_ML

ok


In [24]:
#imp.reload(scr.datapreparation_ML)

In [25]:
for col in pd_annotated.columns:
    print(col,' has the following null values: ', pd_annotated[[col]].isnull().sum())

Unnamed: 0  has the following null values:  Unnamed: 0    0
dtype: int64
Unnamed: 0.1  has the following null values:  Unnamed: 0.1    0
dtype: int64
platform  has the following null values:  platform    0
dtype: int64
title  has the following null values:  title    1
dtype: int64
description  has the following null values:  description    11
dtype: int64
keywords  has the following null values:  keywords    2
dtype: int64
htext  has the following null values:  htext    0
dtype: int64
params  has the following null values:  params    6
dtype: int64
category_regex  has the following null values:  category_regex    0
dtype: int64
wiki  has the following null values:  wiki    373
dtype: int64
category  has the following null values:  category    0
dtype: int64


In [26]:
pd_annotated.shape

(450, 11)

In [27]:
for col in pd_notated.columns:
    print(col,' has the following null values: ', pd_notated[[col]].isnull().sum())

Unnamed: 0  has the following null values:  Unnamed: 0    0
dtype: int64
Unnamed: 0.1  has the following null values:  Unnamed: 0.1    0
dtype: int64
platform  has the following null values:  platform    0
dtype: int64
title  has the following null values:  title    2
dtype: int64
description  has the following null values:  description    9
dtype: int64
keywords  has the following null values:  keywords    3
dtype: int64
htext  has the following null values:  htext    0
dtype: int64
params  has the following null values:  params    13
dtype: int64
category  has the following null values:  category    323
dtype: int64
wiki  has the following null values:  wiki    270
dtype: int64


In [28]:
pd_notated.shape

(323, 10)

In [29]:
pd_annotated['alltext'] = ''
scr.datapreparation_ML.datapreparation(pd_annotated)
pd_notated['alltext'] = ''
scr.datapreparation_ML.datapreparation(pd_notated)

In [30]:
STOP HERE!

SyntaxError: invalid syntax (<ipython-input-30-d5af98942240>, line 1)

# ANALYSIS ROUND 1

## References:
* http://scikit-learn.org/stable/modules/cross_validation.html
* https://stackoverflow.com/questions/24147278/how-do-i-create-test-and-train-samples-from-one-dataframe-with-pandas

## DATA SPLIT

In [ ]:
#from sklearn.model_selection import train_test_split
#xtrain, xtest, ytrain, ytest = train_test_split(pd_annotated.alltext, pd_annotated.category, test_size=0.1, random_state=0)

In [ ]:
train=pd_annotated[['platform', 'alltext','category']].sample(frac=0.75,random_state=0)
test=pd_annotated[['platform', 'alltext','category']].drop(train.index)

In [ ]:
train.head(10)

## DATA MODELLING - VECTOR MODEL 1

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect_r1 = CountVectorizer(ngram_range=(1,2))
X_primercounts = count_vect_r1.fit_transform(train.alltext)
X_primercounts.shape

## DECISION TREE WITH ADABOOST, CLASSIFICATION 1

### References:
* http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier
* http://scikit-learn.org/stable/modules/tree.html
* https://en.wikipedia.org/wiki/AdaBoost
* http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html
* http://scikit-learn.org/stable/auto_examples/ensemble/plot_adaboost_hastie_10_2.html#sphx-glr-auto-examples-ensemble-plot-adaboost-hastie-10-2-py
* https://stackoverflow.com/questions/32210569/using-gridsearchcv-with-adaboost-and-decisiontreeclassifier
* http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_digits.html
* http://scikit-learn.org/stable/tutorial/statistical_inference/model_selection.html

**Also**:
* https://stackoverflow.com/questions/33110973/pass-a-dict-to-scikit-learn-estimator
* http://machinelearningmastery.com/how-to-tune-algorithm-parameters-with-scikit-learn/

### Models

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import zero_one_loss
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
clf_dtm_r1_stump = DecisionTreeClassifier(max_depth=1, min_samples_leaf=1, random_state=0)
clf_dt_r1_stump = clf_dtm_r1_stump.fit(X_primercounts, train.category)

clf_dtm_r1 = DecisionTreeClassifier(min_samples_leaf=2, random_state=0)
clf_dt_r1 = clf_dtm_r1.fit(X_primercounts, train.category)

### Exploration of the Classification

In [ ]:
X_test1counts_stump = count_vect_r1.transform(test.alltext)
predicted_dt_r1_stump = clf_dt_r1_stump.predict(X_test1counts_stump)

In [ ]:
#for platform, category in zip(test.platform, predicted_dt_r1_stump):
#    print('%r => %s' % (platform, category))

In [ ]:
X_test1counts = count_vect_r1.transform(test.alltext)
predicted_dt_r1 = clf_dt_r1.predict(X_test1counts)

In [ ]:
#for platform, category in zip(test.platform, predicted_dt_r1):
#    print('%r => %s' % (platform, category))

### Parameter Searching for the Decision Tree

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Set the parameters by cross-validation
tuned_parameters = [{'max_depth': numpy.arange(5,25,5)}]
scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(clf_dtm_r1, tuned_parameters, cv=2,
                       scoring='%s_macro' % score)
    clf.fit(X_primercounts, train.category)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_predsearch = test.category, clf.predict(X_test1counts)
    print(classification_report(y_true, y_predsearch))
    print()

In [ ]:
X_test1counts_stump = count_vect_r1.transform(test.alltext)
clf_dt_r1_stump_err = 1.0 - clf_dt_r1_stump.score(X_test1counts_stump, test.category)
X_test1count = count_vect_r1.transform(test.alltext)
clf_err = 1.0 - clf.score(X_test1counts, test.category)

In [ ]:
print(clf_dt_r1_stump_err,clf_dt_r1_err)

In [ ]:
print(clf.best_params_)

### AdaBoost

In [ ]:
n_estimators = 40
# A learning rate of 1. may not be optimal for both SAMME and SAMME.R
learning_rate = 1.


#IMPORTANT... https://stackoverflow.com/questions/33110973/pass-a-dict-to-scikit-learn-estimator
clf_dtm_r1.set_params(**clf.best_params_)
clf_dt_r1 = clf_dtm_r1.fit(X_primercounts, train.category)

ada_discrete = AdaBoostClassifier(
    base_estimator = clf_dtm_r1,
    learning_rate=learning_rate,
    n_estimators=n_estimators,
    algorithm="SAMME")
ada_discrete.fit(X_primercounts, train.category)

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot([1, n_estimators], [clf_dt_r1_stump_err] * 2, 'k-',
        label='Decision Stump Error')
ax.plot([1, n_estimators], [clf_dt_r1_err] * 2, 'k--',
        label='Decision Tree Error')

ada_discrete_err = numpy.zeros((n_estimators,))
for i, y_pred in enumerate(ada_discrete.staged_predict(X_test1counts)):
    ada_discrete_err[i] = zero_one_loss(y_pred, test.category)

ada_discrete_err_train = numpy.zeros((n_estimators,))
for i, y_pred in enumerate(ada_discrete.staged_predict(X_primercounts)):
    ada_discrete_err_train[i] = zero_one_loss(y_pred, train.category)

ax.plot(numpy.arange(n_estimators) + 1, ada_discrete_err,
        label='Discrete AdaBoost Test Error',
        color='red')
ax.plot(numpy.arange(n_estimators) + 1, ada_discrete_err_train,
        label='Discrete AdaBoost Train Error',
        color='blue')

ax.set_ylim((0.0, 1.0))
ax.set_xlabel('n_estimators')
ax.set_ylabel('error rate')

leg = ax.legend(loc='upper right', fancybox=True)
leg.get_frame().set_alpha(0.7)

plt.show()

In [ ]:
with open(os.getcwd()+'/data/adaboostmodel1.pkl', 'bw') as mod:
    pickle.dump([train, test, train.alltext, train.category, count_vect_r1, X_primercounts, clf_dtm_r1, clf_dt_r1, ada_discrete, n_estimators, learning_rate], mod)

## Classifying unlabelled records

In [ ]:
X_unclass1counts = count_vect_r1.transform(pd_notated.alltext)
#normalized_X_testround1_counts = sklearn.feature_extraction.text.TfidfTransformer(norm='l2').fit_transform(X_testround1_counts)
predicted_dtada_r1 = ada_discrete.predict(X_unclass1counts)

In [ ]:
for platform, category in zip(pd_notated.platform, predicted_dtada_r1):
    print('%r => %s' % (platform, category))

In [ ]:
with open(os.getcwd()+'/data/abadt_1.csv', 'w') as outfile:
    writer = csv.writer(outfile, delimiter=';', quotechar="'")
    writer.writerow(['platform','assignedcategory','checked','category'])
    for platform, category in zip(pd_notated.platform, predicted_dtada_r1):
        #print('%r => %s' % (platform, category))
        writer.writerow([platform,category,,])

In [ ]:
STOP HERE!!

# POST-ANALYSIS ROUND 1

Data preparation after the manual supervision of the classes

### Python Libraries

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import zero_one_loss
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

## DATA PREPARATION

In [32]:
with open(os.getcwd()+'/data/adaboostmodel1.pkl', 'br') as mod:
    train, test, train.alltext, train.category, count_vect_r1, X_primercounts, clf_dtm_r1, clf_dt_r1, ada_discrete, n_estimators, learning_rate = pickle.load(mod)

In [33]:
test.head()

,platform,alltext,category
0,v4-alpha.getbootstrap.com,buttons world overview content breadcrumb in...,PACKAGE
9,jqueryvalidation.org,still good lots validate options error comes...,PACKAGE
25,api.datamarket.azure.com,microsoft bing azure search v1 composite sal...,PACKAGE
28,preview.c9users.io,arshdkhn1 box chaoz4ngel insert index pollin...,PACKAGE
31,chaijs.com,node org browser delightfully paired asserti...,PACKAGE


In [34]:
X_unclass1counts = count_vect_r1.transform(pd_notated.alltext)

In [35]:
pd_notatedready_r1 = pandas.read_csv(open(os.getcwd()+'/data/abadt_1.csv', 'r'), sep=';', quotechar="'" )

In [36]:
pd_notatedready_r1.loc[pd_notatedready_r1['checked'] != 1, :].shape

(140, 4)

In [37]:
pd_notatedready_r1.shape

(323, 4)

## EVALUATION

In [38]:
print(classification_report(pd_notatedready_r1.loc[pd_notatedready_r1['checked'] != -1, 'category'], pd_notatedready_r1.loc[pd_notatedready_r1['checked'] != -1, 'assignedcategory']))

             precision    recall  f1-score   support

   BUSINESS       0.00      0.00      0.00        14
   COMMUITY       0.00      0.00      0.00        11
       DOCS       0.14      0.14      0.14         7
  ECOMMERCE       0.00      0.00      0.00         5
       NEWS       0.59      0.84      0.69       122
    NOCLASS       0.75      0.14      0.24        21
       PAAS       0.00      0.00      0.00         7
    PACKAGE       0.47      0.70      0.56        70
       REPL       1.00      0.23      0.38        13
    SENGINE       0.00      0.00      0.00         3
      THEME       0.00      0.00      0.00         5
   TRAINING       0.78      0.56      0.65        45

avg / total       0.53      0.57      0.51       323



/home/ec/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# ANALYSIS 2

## RECONSTRUCTING THE DATASETS

In [39]:
pd_notated2 = pd_notated.loc[pd_notatedready_r1['checked'] == -1,:]
pd_notated2.head()

,Unnamed: 0,Unnamed: 0.1,platform,title,description,keywords,htext,params,category,wiki,alltext


In [40]:
pd_annotatedtrans = pd_notated.loc[pd_notatedready_r1['checked'] != -1,:]
pd_annotatedtrans['category'] = pd_notatedready_r1.loc[pd_notatedready_r1['checked'] != -1,'category']

In [41]:
pd_annotatedtrans.head()

,Unnamed: 0,Unnamed: 0.1,platform,title,description,keywords,htext,params,category,wiki,alltext
0,0,0,www.kenneth-truyers.net,my life as a software developer |,blog about development using .net and client-s...,,"Posted on Friday, January 27th, 2017 at 02:42 ...",/2013/04/20/javascript-hoisting-explained/,NEWS,NaN,refactoring blog using html life friday cont...
1,1,1,school.scotch.io,courses ― scotch,,,You trust Scotch tutorials and Scotch authors....,"/build-a-nodejs-website/course-introduction,/b...",TRAINING,NaN,scotch website level author build authors jo...
2,2,2,www.imkrisna.com,i made krisna widhiastra | full-stack developer,,,\n\t\t\t\t\t\t\tCurrently working as Applicati...,/blog/2014/02/if-else-vs-if-if-vs-switch-javas...,NEWS,NaN,full blog malaysia vs professional stack els...
3,3,3,vanilla-js.com,vanilla js,,,The Vanilla JS team maintains every byte of co...,/,PACKAGE,NaN,day js glad framework intuitive maintains us...
4,4,4,www.newmediacampaigns.com,new media campaigns | nmc digital agency,NaN,,Congrats to Hutchison PLLC on being named one ...,/blog/how-to-use-json-apis-with-jquery,BUSINESS,NaN,blog digital hutchison one websites congrats...


In [42]:
pd_annotated2 = pandas.concat([pd_annotated, pd_annotatedtrans], ignore_index = True)
pd_annotated2.head(10)

,Unnamed: 0,Unnamed: 0.1,alltext,category,category_regex,description,htext,keywords,params,platform,title,wiki
0,0,0,css world framework scrollspy buttons group ...,PACKAGE,api|package|framework|librar|stack|licens|addo...,"the most popular html, css, and js framework i...","Bootstrap is the most popular HTML, CSS, and J...",,",/components,/components/breadcrumb/,/migratio...",v4-alpha.getbootstrap.com,"\n \n bootstrap · the most popular html, c...",NaN
1,1,3,iscroll loaded endless ability area current ...,PACKAGE,api|package|framework|librar|stack|licens|addo...,,jScroll is a jQuery plugin for infinite scroll...,,"/,https://github.com/cubiq/iscroll",jscroll.com,jscroll - a jquery plugin for infinite scrolli...,NaN
2,2,10,populate started modeling mongoose html quer...,PACKAGE,api|package|framework|librar|stack|licens|addo...,,Elegant MongoDB object modeling for Node.js Mo...,,"/docs/promises.html,/docs/populate.html,/docs/...",mongoosejs.com,mongoose odm v4.10.5,NaN
3,3,34,stock blog navigator jsonp bar samples react...,PACKAGE,api|package|framework|librar|stack|licens|addo...,Highcharts - Interactive JavaScript charts for...,View demo Get a license,"highcharts, charts, javascript charts, ajax ch...","/samples/data/jsonp.php,/blog/192-use-highchar...",www.highcharts.com,Interactive JavaScript charts for your webpage...,NaN
4,4,41,org php crossorigin en wikipedia welcome htt...,PACKAGE,api|package|framework|librar|stack|licens|addo...,,"\n Welcome to crossorigin.me,...",,"/https://en.wikipedia.org/w/api.php,/,/https:/...",www.crossorigin.me,crossorigin.me,NaN
5,5,47,,PACKAGE,api|package|framework|librar|stack|licens|addo...,,,,NaN,v4-alpha.getbootstrap.com),,NaN
6,6,52,posts quoteson post neville heaven weaver ja...,PACKAGE,api|package|framework|librar|stack|licens|addo...,,Good design goes to heaven; bad design goes ev...,,",/lowell-williams/&quot;},/wp-json/post/,/,/ap...",quotesondesign.com,quotes on design -,NaN
7,7,53,posts io https welcome w github stormconsult...,PACKAGE,api|package|framework|librar|stack|licens|addo...,,"\n Welcome to crossorigin.me,...",,",/http://api.open,/https://en.wikipedia.org/w/...",crossorigin.me,crossorigin.me,NaN
8,8,54,shows html minified events template handle l...,PACKAGE,api|package|framework|librar|stack|licens|addo...,,\nThis demo shows you how to use a simple HTML...,,/,minifiedjs.com,minified.js - a truly lightweight javascript l...,NaN
9,9,59,options lots translations integrate bundled ...,PACKAGE,api|package|framework|librar|stack|licens|addo...,,This jQuery plugin makes simple clientside for...,,/validate/,jqueryvalidation.org,jquery validation plugin | form validation wit...,NaN


## DATA SPLIT

In [43]:
#from sklearn.model_selection import train_test_split
#xtrain, xtest, ytrain, ytest = train_test_split(pd_annotated.alltext, pd_annotated.category, test_size=0.1, random_state=0)

In [44]:
train2=pd_annotated2[['platform', 'alltext','category']].sample(frac=0.75,random_state=0)
test2=pd_annotated2[['platform', 'alltext','category']].drop(train2.index)

In [45]:
train2.head(10)

,platform,alltext,category
369,qnimate.com,web3 ethereum txnscount sendasync eth curren...,TRAINING
628,www.dajonline.net,error404 iframes personal shtml arthur home ...,PACKAGE
401,wind-bow.hyperdev.space,according serves request enough ap https doc...,NOCLASS
14,stormpath.com,know forbidden everything wanted js express ...,PACKAGE
549,blog.sudoask.com,caching nginx happen notes vultr linux speed...,NEWS
278,thenewcode.com,site css vw measurements using author contri...,NEWS
122,blog.serverdensity.com,forbidden javascript automatic conversion ti...,NEWS
741,interactjs.io,standalone inertia interact drop including p...,PACKAGE
386,www.w3schools.com,css templates autocomplete brand shadows hor...,TRAINING
196,www.slant.co,layouts choose js applications want sure hom...,NEWS


## DATA MODELLING - VECTOR MODEL 2

In [46]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect_r2 = CountVectorizer(ngram_range=(1,2))
X_primercounts2 = count_vect_r2.fit_transform(train2.alltext)
X_primercounts2.shape

(580, 18922)

## DECISION TREE WITH ADABOOST, CLASSIFICATION 2

### References:
* http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier
* http://scikit-learn.org/stable/modules/tree.html
* https://en.wikipedia.org/wiki/AdaBoost
* http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html
* http://scikit-learn.org/stable/auto_examples/ensemble/plot_adaboost_hastie_10_2.html#sphx-glr-auto-examples-ensemble-plot-adaboost-hastie-10-2-py
* https://stackoverflow.com/questions/32210569/using-gridsearchcv-with-adaboost-and-decisiontreeclassifier
* http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_digits.html
* http://scikit-learn.org/stable/tutorial/statistical_inference/model_selection.html

**Also**:
* https://stackoverflow.com/questions/33110973/pass-a-dict-to-scikit-learn-estimator
* http://machinelearningmastery.com/how-to-tune-algorithm-parameters-with-scikit-learn/

### Models

In [47]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import zero_one_loss
from sklearn.ensemble import AdaBoostClassifier

In [48]:
clf_dtm_r2_stump = DecisionTreeClassifier(max_depth=1, min_samples_leaf=1, random_state=0)
clf_dt_r2_stump = clf_dtm_r2_stump.fit(X_primercounts2, train2.category)

clf_dtm_r2 = DecisionTreeClassifier(min_samples_leaf=2, random_state=0)
clf_dt_r2 = clf_dtm_r2.fit(X_primercounts2, train2.category)

### Exploration of the Classification

In [49]:
X_test2counts_stump = count_vect_r2.transform(test2.alltext)
predicted_dt_r2_stump = clf_dt_r2_stump.predict(X_test2counts_stump)

In [50]:
#for platform, category in zip(test.platform, predicted_dt_r1_stump):
#    print('%r => %s' % (platform, category))

In [51]:
X_test2counts = count_vect_r2.transform(test2.alltext)
predicted_dt_r2 = clf_dt_r2.predict(X_test2counts)

In [58]:
for plat, category in zip(zip(test2.platform,test2.category), predicted_dt_r2):
    print('%r => %s => %s' % (plat[0], plat[1], category))

'jqueryvalidation.org' => PACKAGE => PACKAGE
'www.bootstraptoggle.com' => PACKAGE => PACKAGE
'ajax.googleapis.com' => PACKAGE => BUSINESS
'mathjs.org' => PACKAGE => PACKAGE
'lesscss.org' => PACKAGE => NOCLASS
'api.datamarket.azure.com' => PACKAGE => COMMUITY
'preview.c9users.io' => PACKAGE => NEWS
'mojs.io' => PACKAGE => PACKAGE
'ipinfo.io' => PACKAGE => PACKAGE
'materializecss.com' => PACKAGE => PACKAGE
'bootstrap-switch.min.css' => PACKAGE => NOCLASS
'fittextjs.com' => PACKAGE => DOCS
'paperjs.org' => PACKAGE => NOCLASS
'd3js.org' => PACKAGE => PACKAGE
'www.npmjs.com' => PACKAGE => PAAS
'www.wikidata.org' => PACKAGE => PACKAGE
'www.oneskyapp.com' => PACKAGE => DOCS
'jsonip.com' => PACKAGE => PACKAGE
'cdnjs.com' => PACKAGE => NEWS
'npmcdn.com' => PACKAGE => PACKAGE
'expressjs.com' => PACKAGE => PACKAGE
'packagecontrol.io' => PACKAGE => PACKAGE
'getbootstrap.com.br' => PACKAGE => PACKAGE
'howtocenterincss.com' => PACKAGE => NOCLASS
'query.yahooapis.com' => PACKAGE => PACKAGE
'www.boots

### Parameter Searching for the Decision Tree

In [53]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Set the parameters by cross-validation
tuned_parameters = [{'max_depth': numpy.arange(5,25,2)}]
scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(clf_dtm_r2, tuned_parameters, cv=2,
                       scoring='%s_macro' % score)
    clf.fit(X_primercounts2, train2.category)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_predsearch = test2.category, clf.predict(X_test2counts)
    print(classification_report(y_true, y_predsearch))
    print()

# Tuning hyper-parameters for precision



/home/ec/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ec/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ec/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ec/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ec/anaconda3/lib/python3.5/sit

Best parameters set found on development set:

{'max_depth': 23}

Grid scores on development set:

0.217 (+/-0.102) for {'max_depth': 5}
0.209 (+/-0.000) for {'max_depth': 7}
0.186 (+/-0.063) for {'max_depth': 9}
0.220 (+/-0.074) for {'max_depth': 11}
0.190 (+/-0.022) for {'max_depth': 13}
0.183 (+/-0.030) for {'max_depth': 15}
0.197 (+/-0.050) for {'max_depth': 17}
0.190 (+/-0.032) for {'max_depth': 19}
0.285 (+/-0.087) for {'max_depth': 21}
0.288 (+/-0.081) for {'max_depth': 23}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

             precision    recall  f1-score   support

   BUSINESS       0.15      0.22      0.18         9
   COMMUITY       0.00      0.00      0.00         1
       DOCS       0.00      0.00      0.00         4
  ECOMMERCE       0.00      0.00      0.00         1
       NEWS       0.69      0.69      0.69        81
    NOCLASS       0.33      0.30      0.32        10
     

/home/ec/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [54]:
X_test2counts_stump = count_vect_r2.transform(test2.alltext)
clf_dt_r2_stump_err = 1.0 - clf_dt_r2_stump.score(X_test2counts_stump, test2.category)
X_test2count = count_vect_r2.transform(test2.alltext)
clf_dt_r2_err = 1.0 - clf.score(X_test2counts, test2.category)

In [55]:
print(clf_dt_r2_stump_err,clf_dt_r2_err)

0.580310880829 0.760996437808


In [56]:
print(clf.best_params_)

{'max_depth': 23}


### AdaBoost

In [57]:
n_estimators = 40
# A learning rate of 1. may not be optimal for both SAMME and SAMME.R
learning_rate = .5


#IMPORTANT... https://stackoverflow.com/questions/33110973/pass-a-dict-to-scikit-learn-estimator
clf_dtm_r2.set_params(**clf.best_params_)
clf_dt_r2 = clf_dtm_r2.fit(X_primercounts2, train2.category)

ada_discrete = AdaBoostClassifier(
    base_estimator = clf_dtm_r2,
    learning_rate=learning_rate,
    n_estimators=n_estimators,
    algorithm="SAMME")
ada_discrete.fit(X_primercounts2, train2.category)

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot([1, n_estimators], [clf_dt_r2_stump_err] * 2, 'k-',
        label='Decision Stump Error')
ax.plot([1, n_estimators], [clf_dt_r2_err] * 2, 'k--',
        label='Decision Tree Error')

ada_discrete_err = numpy.zeros((n_estimators,))
for i, y_pred in enumerate(ada_discrete.staged_predict(X_test2counts)):
    ada_discrete_err[i] = zero_one_loss(y_pred, test2.category)

ada_discrete_err_train = numpy.zeros((n_estimators,))
for i, y_pred in enumerate(ada_discrete.staged_predict(X_primercounts2)):
    ada_discrete_err_train[i] = zero_one_loss(y_pred, train2.category)

ax.plot(numpy.arange(n_estimators) + 1, ada_discrete_err,
        label='Discrete AdaBoost Test Error',
        color='red')
ax.plot(numpy.arange(n_estimators) + 1, ada_discrete_err_train,
        label='Discrete AdaBoost Train Error',
        color='blue')

ax.set_ylim((0.0, 1.0))
ax.set_xlabel('n_estimators')
ax.set_ylabel('error rate')

leg = ax.legend(loc='upper right', fancybox=True)
leg.get_frame().set_alpha(0.7)

plt.show()

AttributeError: 'tuple' object has no attribute 'figure'

In [ ]:
Stop HERE!

In [ ]:
with open(os.getcwd()+'/data/adaboostmodel1.pkl', 'bw') as mod:
    pickle.dump([train, test, train.alltext, train.category, count_vect_r1, X_primercounts, clf_dtm_r1, clf_dt_r1, ada_discrete, n_estimators, learning_rate], mod)

## Classifying unlabelled records

In [ ]:
X_unclass1counts = count_vect_r1.transform(pd_notated.alltext)
#normalized_X_testround1_counts = sklearn.feature_extraction.text.TfidfTransformer(norm='l2').fit_transform(X_testround1_counts)
predicted_dtada_r1 = ada_discrete.predict(X_unclass1counts)

In [ ]:
for platform, category in zip(pd_notated.platform, predicted_dtada_r1):
    print('%r => %s' % (platform, category))

In [ ]:
with open(os.getcwd()+'/data/abadt_1.csv', 'w') as outfile:
    writer = csv.writer(outfile, delimiter=';', quotechar="'")
    writer.writerow(['platform','assignedcategory','checked','category'])
    for platform, category in zip(pd_notated.platform, predicted_dtada_r1):
        #print('%r => %s' % (platform, category))
        writer.writerow([platform,category,,])

In [ ]:
here stop

# DUMPED!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [59]:
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_predict
from sklearn import metrics

## RANDOM FOREST, CLASSIFICATION 1

### References:
* http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
* http://scikit-learn.org/stable/modules/ensemble.html#forest

### Models

In [60]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import zero_one_loss

In [61]:
#n_estimators = 25
# A learning rate of 1. may not be optimal for both SAMME and SAMME.R
#learning_rate = 1.

clf_rfm_r2_stump = RandomForestClassifier(max_depth=1, min_samples_leaf=1, n_estimators=n_estimators, random_state=0)
clf_rf_r2_stump = clf_rfm_r2_stump.fit(X_primercounts2, train2.category)

clf_rfm_r2 = RandomForestClassifier(max_depth=20, min_samples_leaf=2, n_estimators=n_estimators, random_state=0)
clf_rf_r2 = clf_rfm_r2.fit(X_primercounts2, train2.category)

### Classification

In [62]:
#X_test1counts_stump = count_vect_r2.transform(test.alltext)
predicted_rf_r2_stump = clf_rf_r2_stump.predict(X_test2counts_stump)

In [63]:
for platform, category in zip(test2.platform, predicted_rf_r2_stump):
    print('%r => %s' % (platform, category))

'jqueryvalidation.org' => NEWS
'www.bootstraptoggle.com' => NEWS
'ajax.googleapis.com' => NEWS
'mathjs.org' => NEWS
'lesscss.org' => NEWS
'api.datamarket.azure.com' => NEWS
'preview.c9users.io' => NEWS
'mojs.io' => NEWS
'ipinfo.io' => NEWS
'materializecss.com' => NEWS
'bootstrap-switch.min.css' => NEWS
'fittextjs.com' => NEWS
'paperjs.org' => NEWS
'd3js.org' => NEWS
'www.npmjs.com' => NEWS
'www.wikidata.org' => NEWS
'www.oneskyapp.com' => NEWS
'jsonip.com' => NEWS
'cdnjs.com' => NEWS
'npmcdn.com' => NEWS
'expressjs.com' => NEWS
'packagecontrol.io' => NEWS
'getbootstrap.com.br' => NEWS
'howtocenterincss.com' => NEWS
'query.yahooapis.com' => NEWS
'www.bootstrapcdn.com' => NEWS
'www.geoplugin.net' => NEWS
'ip-api.com' => NEWS
'scrollrevealjs.org' => NEWS
'medium.com' => NEWS
'sixrevisions.com' => NEWS
'recurial.com' => NEWS
'jhtmlcss.blogspot.com' => NEWS
'wildermuth.com' => NEWS
'codecondo.com' => NEWS
'lab.abhinayrathore.com' => NEWS
'www.standardista.com' => NEWS
'quotes.stormconsultan

In [64]:
#X_test2counts = count_vect_r2.transform(test2.alltext)
predicted_rf_r2 = clf_rf_r2.predict(X_test2counts)

In [65]:
best_categories = []
for plat, category1, category2 in zip(zip(test2.platform,test2.category), predicted_rf_r2, predicted_dt_r2):
    best_category = ''
    if category1 == category2:
        best_category = category1
    else:
        if category1 == 'NEWS' and category2 == 'TRAINING':
            best_category = category1

        else:
            best_category = category2
    
    if category1 != category2 and len(plat[0].split('.')) > 1:
        if category2 in ['NOCLASS', 'BUSINESS', 'DOCS', 'PAAS', 'NEWS']:
            #print(plat[0])
            if len(plat[0].split('.')[-2]) > 2:
                #print(plat[0].split('.')[-2])
                if plat[0].split('.')[-2][-2:] == 'js':
                    #print(len(plat[0].split('.')))
                    #print(plat[0].split('.')[-2])
                    best_category = 'PACKAGE'
    
    best_categories.append(best_category)
    print('%r => %s => %s => %s => %s' % (plat[0], plat[1].lower(), category1, category2, best_category.lower()))

'jqueryvalidation.org' => package => NEWS => PACKAGE => package
'www.bootstraptoggle.com' => package => NEWS => PACKAGE => package
'ajax.googleapis.com' => package => NEWS => BUSINESS => business
'mathjs.org' => package => PACKAGE => PACKAGE => package
'lesscss.org' => package => NEWS => NOCLASS => noclass
'api.datamarket.azure.com' => package => NEWS => COMMUITY => commuity
'preview.c9users.io' => package => NEWS => NEWS => news
'mojs.io' => package => NEWS => PACKAGE => package
'ipinfo.io' => package => PACKAGE => PACKAGE => package
'materializecss.com' => package => NEWS => PACKAGE => package
'bootstrap-switch.min.css' => package => NEWS => NOCLASS => noclass
'fittextjs.com' => package => NEWS => DOCS => package
'paperjs.org' => package => NEWS => NOCLASS => package
'd3js.org' => package => PACKAGE => PACKAGE => package
'www.npmjs.com' => package => PACKAGE => PAAS => package
'www.wikidata.org' => package => NEWS => PACKAGE => package
'www.oneskyapp.com' => package => NEWS => DOCS =

In [66]:
#X_test2counts_stump = count_vect_r2.transform(test2.alltext)
clf_rf_r2_stump_err = 1.0 - clf_rf_r2_stump.score(X_test2counts_stump, test2.category)
#X_test2count = count_vect_r2.transform(test2.alltext)
clf_rf_r2_err = 1.0 - clf_rf_r2.score(X_test2counts, test2.category)

clf_rf_r2_bc_err = 1.0 - clf_rf_r2.score(X_test2counts, best_categories)

In [67]:
print(clf_rf_r2_stump_err,clf_rf_r2_err, clf_rf_r2_bc_err)

0.580310880829 0.502590673575 0.445595854922


In [68]:
print(classification_report(test2.category, best_categories))

             precision    recall  f1-score   support

   BUSINESS       0.25      0.33      0.29         9
   COMMUITY       0.00      0.00      0.00         1
       DOCS       0.00      0.00      0.00         4
  ECOMMERCE       0.00      0.00      0.00         1
       NEWS       0.67      0.77      0.72        81
    NOCLASS       0.21      0.30      0.25        10
       PAAS       0.00      0.00      0.00         1
    PACKAGE       0.64      0.70      0.67        46
       REPL       0.50      0.14      0.22         7
      THEME       0.50      0.11      0.18         9
   TRAINING       0.86      0.25      0.39        24

avg / total       0.61      0.56      0.55       193



/home/ec/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### AdaBoost

In [69]:
n_estimators = 40
# A learning rate of 1. may not be optimal for both SAMME and SAMME.R
learning_rate = .2


#IMPORTANT... https://stackoverflow.com/questions/33110973/pass-a-dict-to-scikit-learn-estimator
#clf_dtm_r2.set_params(**clf.best_params_)
#clf_dt_r2 = clf_dtm_r2.fit(X_primercounts2, train2.category)

ada_discrete = AdaBoostClassifier(
    base_estimator = clf_rf_r2,
    learning_rate=learning_rate,
    n_estimators=n_estimators,
    algorithm="SAMME")
ada_discrete.fit(X_primercounts2, train2.category)

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot([1, n_estimators], [clf_rf_r2_stump_err] * 2, 'k-',
        label='Decision Stump Error')
ax.plot([1, n_estimators], [clf_rf_r2_err] * 2, 'k--',
        label='Decision Tree Error')

ada_discrete_err = numpy.zeros((n_estimators,))
for i, y_pred in enumerate(ada_discrete.staged_predict(X_test2counts)):
    ada_discrete_err[i] = zero_one_loss(y_pred, test2.category)

ada_discrete_err_train = numpy.zeros((n_estimators,))
for i, y_pred in enumerate(ada_discrete.staged_predict(X_primercounts2)):
    ada_discrete_err_train[i] = zero_one_loss(y_pred, train2.category)

ax.plot(numpy.arange(n_estimators) + 1, ada_discrete_err,
        label='Discrete AdaBoost Test Error',
        color='red')
ax.plot(numpy.arange(n_estimators) + 1, ada_discrete_err_train,
        label='Discrete AdaBoost Train Error',
        color='blue')

ax.set_ylim((0.0, 1.0))
ax.set_xlabel('n_estimators')
ax.set_ylabel('error rate')

leg = ax.legend(loc='upper right', fancybox=True)
leg.get_frame().set_alpha(0.7)

plt.show()

AttributeError: 'tuple' object has no attribute 'figure'

In [71]:
predicted_ada_r2 = ada_discrete.predict(X_test2counts)

In [89]:
newbest_categories = []
for *bcategory, category in zip(zip(zip(test2.platform,test2.category), best_categories), predicted_ada_r2):
    plat = bcategory[0][0]
    bcategory = bcategory[0][1]
    if bcategory == 'TRAINING' and category != 'NOCLASS':
        newbest_categories.append('TRAINING')
    else:
        newbest_categories.append(category)
    
    if category == 'REPL':
        print('%r => %s => %s => %s' % (plat[0], plat[1], bcategory.lower(), category))

'jshint.com' => REPL => news => REPL
'university.mongodb.com' => TRAINING => training => REPL
'www.punkchip.com' => NEWS => news => REPL
'genius.com' => NEWS => news => REPL
'jsfidle.net' => REPL => repl => REPL


In [85]:
print(classification_report(test2.category, predicted_ada_r2))

             precision    recall  f1-score   support

   BUSINESS       0.00      0.00      0.00         9
   COMMUITY       0.00      0.00      0.00         1
       DOCS       1.00      0.25      0.40         4
  ECOMMERCE       0.00      0.00      0.00         1
       NEWS       0.70      0.77      0.73        81
    NOCLASS       0.20      0.30      0.24        10
       PAAS       0.00      0.00      0.00         1
    PACKAGE       0.57      0.85      0.68        46
       REPL       0.40      0.29      0.33         7
      THEME       1.00      0.11      0.20         9
   TRAINING       0.79      0.46      0.58        24

avg / total       0.62      0.62      0.58       193



/home/ec/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [86]:
print(classification_report(test2.category, newbest_categories))

             precision    recall  f1-score   support

   BUSINESS       0.00      0.00      0.00         9
   COMMUITY       0.00      0.00      0.00         1
       DOCS       1.00      0.25      0.40         4
  ECOMMERCE       0.00      0.00      0.00         1
       NEWS       0.70      0.77      0.73        81
    NOCLASS       0.20      0.30      0.24        10
       PAAS       0.00      0.00      0.00         1
    PACKAGE       0.57      0.83      0.67        46
       REPL       0.50      0.29      0.36         7
      THEME       1.00      0.11      0.20         9
   TRAINING       0.76      0.54      0.63        24

avg / total       0.62      0.62      0.59       193



/home/ec/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
